In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
pd.options.plotting.backend = "plotly"

In [2]:
class Environnement:
    def __init__(self,nb_client,facets,assets,periode_collecte):
        self.nb_client = nb_client
        self.facets = facets
        self.assets = assets
        self.periode_collecte = periode_collecte
        self.facets_par_min = facets * 60 / periode_collecte
        self.assets_par_min = assets * 60 / periode_collecte
        self.données_par_min = self.facets_par_min + self.assets_par_min


class Application:
    def __init__(self,CPU,RAM,SSD ):
        self.CPU = CPU
        self.RAM = RAM
        self.SSD = SSD


class emission:
    def __init__(self,cassandra,kafka,aks,nb_client, d_vie_serv, d_vie_sys, st_data_p, replication, nb_metrique, intensite_ssd):
        self.cassandra = cassandra
        self.kafka = kafka
        self.aks = aks
        self.total = cassandra + kafka + aks
        self.total_unit = self.total / nb_client
        self.total_acv = self.total * d_vie_sys / d_vie_serv
        self.total_acv_unit = self.total_acv / nb_client
        self.st_annuel = (st_data_p * 365 * 24 * 60 * nb_metrique * 1e-12) * intensite_ssd * replication *  Duree_vie_serveur * nb_client * sum(i  for i in range(0,int(Duree_vie_systeme / Duree_vie_serveur)+1))
        self.total_acv2 = self.total_acv + self.st_annuel
        self.total_acv2_unit = self.total_acv2 / nb_client

#def calcul_CO2(cpu, ram, ssd):
#    return cpu * intensite.CPU + ram * intensite.RAM + ssd * intensite.SSD
        
         
intensite = Application(19.7,3.75,51)
varta = {
    "data" : Environnement(27000,14,0,120),
    "cassandra" : Application(170,960,66),
    "kafka" : Application(15,96,26),
    "aks" : Application(35,960,2)
}

HEMS = {
    "data" : Environnement(1,36,207,15),
}

Duree_vie_serveur = 4
Duree_vie_systeme = 30
st_data_p = 8.5
replication = 5

def Stock_annuel(Env):
    return (st_data_p * 365 * 24 * 60 * Env["data"].données_par_min * 1e-12) * intensite.SSD * replication *  Duree_vie_serveur * Env["data"].nb_client * sum(i  for i in range(0,int(Duree_vie_systeme / Duree_vie_serveur)+1))


facteur_asset_facet = 0.5

In [3]:
df=pd.DataFrame(
    {
        "Application" : pd.Categorical(["Cassandra", "Cassandra", "Cassandra", "Kafka", "Kafka", "Kafka", "Aks", "Aks", "Aks"]),
        "Infra" : pd.Categorical(["CPU", "RAM", "SSD", "CPU", "RAM", "SSD", "CPU", "RAM", "SSD"]),
        "Provision" : [varta["cassandra"].CPU, varta["cassandra"].RAM, varta["cassandra"].SSD, varta["kafka"].CPU, varta["kafka"].RAM, varta["kafka"].SSD, varta["aks"].CPU, varta["aks"].RAM, varta["aks"].SSD],
        "Intensite_par_infra" : [intensite.CPU, intensite.RAM, intensite.SSD, intensite.CPU, intensite.RAM, intensite.SSD, intensite.CPU, intensite.RAM, intensite.SSD]
    }
)

df=df.assign(Intensite_serv=df["Provision"] * df["Intensite_par_infra"])
df=df.assign(Intensite_syst=df["Intensite_serv"] * Duree_vie_systeme  / Duree_vie_serveur)
df.loc[(df["Application"]=="Cassandra") & (df["Infra"]=="SSD"), "Intensite_syst"] = df['Intensite_syst'] + Stock_annuel(varta)
df=df.assign(Intensite_syst_unit=df["Intensite_syst"] / varta["data"].nb_client)



df=df.assign(infra_metrique=df["Provision"] / (varta["data"].données_par_min * varta["data"].nb_client ))
df=df.assign(Intensite_HEMS_serv= HEMS["data"].facets_par_min * HEMS["data"].nb_client * df["infra_metrique"] * df["Intensite_par_infra"])

df.loc[df["Application"]=="Cassandra", "Intensite_HEMS_serv"] = df['Intensite_HEMS_serv'] + HEMS["data"].assets_par_min * HEMS["data"].nb_client * df["infra_metrique"] * df["Intensite_par_infra"]
df.loc[df["Application"]!="Cassandra", "Intensite_HEMS_serv"] = df['Intensite_HEMS_serv'] + HEMS["data"].assets_par_min * facteur_asset_facet * HEMS["data"].nb_client * df["infra_metrique"] * df["Intensite_par_infra"]

df=df.assign(Intensite_HEMS_syst=df["Intensite_HEMS_serv"] * Duree_vie_systeme  / Duree_vie_serveur)
df.loc[(df["Application"]=="Cassandra") & (df["Infra"]=="SSD"), "Intensite_HEMS_syst"] = df['Intensite_HEMS_syst'] + Stock_annuel(HEMS)
df=df.assign(Intensite_HEMS_syst_unit=df["Intensite_HEMS_syst"] / HEMS["data"].nb_client)

df=round(df,2)

fig = px.sunburst(df, path=['Application', 'Infra'], values='Intensite_syst_unit')
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [4]:
df

,Application,Infra,Provision,Intensite_par_infra,Intensite_serv,Intensite_syst,Intensite_syst_unit,infra_metrique,Intensite_HEMS_serv,Intensite_HEMS_syst,Intensite_HEMS_syst_unit
0,Cassandra,CPU,170,19.70,3349.0,25117.50,0.93,0.00,17.22,129.18,129.18
1,Cassandra,RAM,960,3.75,3600.0,27000.00,1.00,0.01,18.51,138.86,138.86
2,Cassandra,SSD,66,51.00,3366.0,49360.39,1.83,0.00,17.31,253.85,253.85
3,Kafka,CPU,15,19.70,295.5,2216.25,0.08,0.00,0.87,6.54,6.54
4,Kafka,RAM,96,3.75,360.0,2700.00,0.10,0.00,1.06,7.97,7.97
5,Kafka,SSD,26,51.00,1326.0,9945.00,0.37,0.00,3.91,29.36,29.36
6,Aks,CPU,35,19.70,689.5,5171.25,0.19,0.00,2.04,15.27,15.27
7,Aks,RAM,960,3.75,3600.0,27000.00,1.00,0.01,10.63,79.71,79.71
8,Aks,SSD,2,51.00,102.0,765.00,0.03,0.00,0.30,2.26,2.26


In [5]:
fig = px.sunburst(df, path=['Application', 'Infra'], values='Intensite_HEMS_syst_unit')

fig.update_traces(textinfo="label+percent entry+value")
fig.show()

In [6]:
df['Intensite_HEMS_syst'].sum()

663.0

In [7]:
Stock_annuel(HEMS)

124.02200563199999

In [8]:
class acv:
    def __init__(self,emissions,ddv,ddv_syst, site ):
        self.emissions = emissions
        self.ddv = ddv
        self.nb_ddv_syst = ddv_syst / ddv
        self.site = site
        self.emissions_ddv = emissions * self.nb_ddv_syst * site

Intensite_mix_elec = 0.05 #Kg CO2 eq / Kwh
Ratio_Fab = 0.45
Ratio_Use = 1-Ratio_Fab

Pv = acv(1000,30,30,1)
Bat= acv(173,5000,365*30,5)
Inverter_Pv =  acv(140,15,30,Pv.site)
Inverter_Bat =  acv(140,15,30,Bat.site / 3)
Gateway_fab = acv(50,5,30,1)
Gateway_use = acv(Intensite_mix_elec,1,30*365*24,0.005) #acv(intensite du mix elec, 1, Nombre d'heure en 30 ans, Puissance en KW)
Server_fab = acv(df['Intensite_HEMS_syst_unit'].sum(),30,30,1) 
Server_use = acv(df['Intensite_HEMS_syst_unit'].sum() * Ratio_Use / Ratio_Fab, 30, 30, 1)


    
    

In [9]:
df_CO2 = pd.DataFrame(
    {
        'Device' : pd.Categorical(['Pv', 'Bat', 'Inverter', 'Inverter', 'Gateway', 'Gateway', 'Server', 'Server','Instal']),
        'Family_device' : pd.Categorical(['Pv','Bat','Pv','Bat','Gateway','Gateway','Server','Server','Instal']),
        'Post_emission' : pd.Categorical(['Fab','Fab','Fab','Fab','Fab','Use','Fab','Use','Instal'])
    }
    
)
df_CO2['Emissions'] = [Pv.emissions_ddv, Bat.emissions_ddv, Inverter_Pv.emissions_ddv, Inverter_Bat.emissions_ddv, Gateway_fab.emissions_ddv,Gateway_use.emissions_ddv,Server_fab.emissions_ddv,Server_use.emissions_ddv,0]
df_CO2.loc[(df_CO2["Device"]=="Instal") , "Emissions"] = df_CO2['Emissions'].sum() * 0.1 / 0.9
df_CO2=round(df_CO2,2)

In [10]:
df_CO2

,Device,Family_device,Post_emission,Emissions
0,Pv,Pv,Fab,1000.00
1,Bat,Bat,Fab,1894.35
2,Inverter,Pv,Fab,280.00
3,Inverter,Bat,Fab,466.67
4,Gateway,Gateway,Fab,300.00
5,Gateway,Gateway,Use,65.70
6,Server,Server,Fab,663.00
7,Server,Server,Use,810.33
8,Instal,Instal,Instal,608.89


In [11]:
fig = px.sunburst(df_CO2, path=['Family_device', 'Device', 'Post_emission'], values='Emissions')

fig.update_traces(textinfo="label+percent entry")
fig.show()

In [12]:
df_CO2['Emissions'].sum()

6088.9400000000005

In [13]:
df_CO2.loc[(df_CO2["Device"]=="Server") , "Emissions"].sum()

1473.33

In [14]:
Stock_annuel(varta)/27000

0.8931625919999998